# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data1 = data[['CustomerID', 'ProductName', 'Quantity']].groupby(['CustomerID','ProductName']).sum()
data1.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data2 = pd.pivot_table(data1, values='Quantity', index='ProductName', columns=['CustomerID'])
data2 = data2.fillna(0)
data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform

distance = pdist(data2, metric='euclidean')
print('pdist= ', '\n', distance , '\n')
distance2 = squareform(distance)
print('squareform= ', '\n', distance2)


pdist=  
 [279.99642855 253.66907577 238.5120542  ... 264.14200726 268.28343221
 273.15014186] 

squareform=  
 [[  0.         279.99642855 253.66907577 ... 255.67166444 258.0910692
  278.47800631]
 [279.99642855   0.         271.49953959 ... 265.7367118  279.55500353
  286.80655502]
 [253.66907577 271.49953959   0.         ... 245.27535547 253.01580978
  269.43644891]
 ...
 [255.67166444 265.7367118  245.27535547 ...   0.         264.14200726
  268.28343221]
 [258.0910692  279.55500353 253.01580978 ... 264.14200726   0.
  273.15014186]
 [278.47800631 286.80655502 269.43644891 ... 268.28343221 273.15014186
    0.        ]]


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
import numpy as np
data4 = pd.DataFrame(1 / (1 + squareform(pdist(data2.T, metric='euclidean'))), index=data2.columns, columns=data2.columns)
np.fill_diagonal(data4.values, 0)
display(data4)
sim = list(data4[33].sort_values(ascending=False)[:-1].head(5).index)
sim

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,0.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,0.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,0.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,0.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,0.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,0.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,0.000000,0.003277,0.003266


[264, 3317, 3535, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
data5 = data1.loc[sim]
data5

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
data6 = data5.groupby(['ProductName']).sum()
data6 = data6.sort_values(by='Quantity', ascending=False)
data6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
data7 = data2[[33]].merge(data6, right_on='ProductName', left_on='ProductName')
data7 = data7.sort_values(['Quantity'], ascending=False)
data7 = data7[data7[33] == 0].sort_values(by='Quantity', ascending=False)
data7.head(5)

,33,Quantity
ProductName,,
Butter - Unsalted,0.0,3
Soup - Campbells Bean Medley,0.0,3
Wine - Blue Nun Qualitatswein,0.0,3
Wine - Ej Gallo Sierra Valley,0.0,3
Beans - Wax,0.0,2


In [11]:
lst_pr = list(data7.index)[:5]
lst_pr

['Butter - Unsalted',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Wine - Ej Gallo Sierra Valley',
 'Beans - Wax']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [14]:
dicti = {}
custList = list(set(data['CustomerID']))
for cust in custList:
    simil = list(data4[cust].sort_values(ascending=False)[:-1].head(5).index)
    similP = data1.loc[simil]
    groupP = similP.groupby(['ProductName']).sum()
    groupP = groupP.sort_values(by='Quantity', ascending=False)
    merge = data2[[cust]].merge(groupP, right_on='ProductName', left_on='ProductName')
    merge = merge.sort_values(['Quantity'], ascending=False)
    merge = merge[merge[cust] == 0].sort_values(by='Quantity', ascending=False)
    listP = list(merge.index)[:5]
    dicti[cust] = listP

In [ ]:
#dicti

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [20]:
dfRec = pd.DataFrame.from_dict(dicti, orient='index').reset_index()
dfRec.columns = [['customerID', 'p1', 'p2', 'p3', 'p4', 'p5']]
dfRec.head()

,customerID,p1,p2,p3,p4,p5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,59399,"Shrimp - Baby, Warm Water",Towels - Paper / Kraft,Knife Plastic - White,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
2,92168,Cup - Translucent 7 Oz Clear,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Spinach - Baby,Pop Shoppe Cream Soda
3,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
4,18441,Soupfoamcont12oz 112con,Beef - Texas Style Burger,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [25]:
data10 = pd.DataFrame(1 / (1 + squareform(pdist(data2.T, metric='correlation'))), index=data2.columns, columns=data2.columns)
data10.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
